# Milestone Project 2 - Blackjack Game
In this milestone project you will be creating a Complete BlackJack Card Game in Python.

Here are the requirements:

* You need to create a simple text-based [BlackJack](https://en.wikipedia.org/wiki/Blackjack) game
* The game needs to have one player versus an automated dealer.
* The player can stand or hit.
* The player must be able to pick their betting amount.
* You need to keep track of the player's total money.
* You need to alert the player of wins, losses, or busts, etc...

And most importantly:

* **You must use OOP and classes in some portion of your game. You can not just use functions in your game. Use classes to help you define the Deck and the Player's hand. There are many right ways to do this, so explore it well!**


Feel free to expand this game. Try including multiple players. Try adding in Double-Down and card splits! Remember to you are free to use any resources you want and as always:

# HAVE FUN!

In [79]:
# https://tekeye.uk/playing_cards/svg-playing-cards

from IPython.display import display, clear_output
from ipywidgets import Button, Layout, BoundedIntText, HBox, VBox, GridBox, HTML
import random
import time


card_deck = {
    "clubs_2": 2,
    "clubs_3": 3,
    "clubs_4": 4,
    "clubs_5": 5,
    "clubs_6": 6,
    "clubs_7": 7,
    "clubs_8": 8,
    "clubs_9": 9,
    "clubs_10": 10,
    "clubs_ace": 11,
    "clubs_jack": 10,
    "clubs_king": 10,
    "clubs_queen": 10,
    "diamonds_2": 2,
    "diamonds_3": 3,
    "diamonds_4": 4,
    "diamonds_5": 5,
    "diamonds_6": 6,
    "diamonds_7": 7,
    "diamonds_8": 8,
    "diamonds_9": 9,
    "diamonds_10": 10,
    "diamonds_ace": 11,
    "diamonds_jack": 10,
    "diamonds_king": 10,
    "diamonds_queen": 10,
    "hearts_2": 2,
    "hearts_3": 3,
    "hearts_4": 4,
    "hearts_5": 5,
    "hearts_6": 6,
    "hearts_7": 7,
    "hearts_8": 8,
    "hearts_9": 9,
    "hearts_10": 10,
    "hearts_ace": 11,
    "hearts_jack": 10,
    "hearts_king": 10,
    "hearts_queen": 10,
    "spades_2": 2,
    "spades_3": 3,
    "spades_4": 4,
    "spades_5": 5,
    "spades_6": 6,
    "spades_7": 7,
    "spades_8": 8,
    "spades_9": 9,
    "spades_10": 10,
    "spades_ace": 11,
    "spades_jack": 10,
    "spades_king": 10,
    "spades_queen": 10
}

dealer = None
players = None
number_of_players = None
bets_input = None
name_input = None
active_player = None
actice_hand = None
dealer_title_html = '<p style="font-size: 16px; font-weight: bold; line-height: 16px;">Dealer</p>'
short_pause = 0.8
long_pause = 1.8

    
class Hand:
    def __init__(self):
        self.cards = []
        self.value = 0
        self.bet = 0
        self.blackjack = False
        self.finished_round = False
        
    def __lt__(self,other):
        return self.value < other.value
        
    def __gt__(self,other):
        return self.value > other.value
        
    def __eq__(self,other):
        return self.value == other.value
    
    def addToHand(self, cards):
        self.cards += cards
        for card in cards:
            self.value += card_deck[card]
        if self.value > 21:
            value = 0
            number_of_aces = 0
            for card in self.cards:
                if "ace" in card:
                    number_of_aces += 1
                value += card_deck[card]
            for i in range(number_of_aces):
                if value > 21:
                    value -= 10
                else:
                    break
            self.value = value
        
    def display(self, end = None):
        card_width = 140
        cards_in_row = 4
        hand_html = f'<div style="display: grid; gap: 8px; grid-template-columns: repeat({cards_in_row},'\
        f'{card_width}px); justify-content: center;">'
        for card in self.cards[:end]:
            hand_html += '<div style="background-image:url(https://tekeye.uk/playing_cards/images/svg_playing_cards/'\
            f'fronts/{card}.svg); background-size: contain; background-repeat: no-repeat; height:200px;"></div>'
        if end is not None:
            hand_html += '<div style="background-image:url(https://tekeye.uk/playing_cards/images/svg_playing_cards/'\
            'backs/blue2.svg); background-size: contain; background-repeat: no-repeat; height:200px;"></div>' * \
            (len(self.cards) - end)
        hand_html += '</div>'
        return hand_html
        
    
class winLose:
    def lose(self, money):
        self.money -= money
        
    def win(self, money):
        self.money += money
        
    
class Dealer(winLose):
    def __init__(self, money = 1_000):
        self.hand = Hand()
        self.money = money
        self.deck = Deck()
    
    def clear_hand(self):
        self.hand = Hand()

        
class Player(winLose):
    def __init__(self, name, money = 100):
        self.hands = [Hand()]
        self.money = money
        self.name = name
        
    def clear_hands(self):
        self.hands = [Hand()]
        
    
class Deck:
    number_of_decks = 6
    
    def __init__(self):
        self.deck = list(card_deck.keys()) * self.number_of_decks
        random.shuffle(self.deck)
    
    def shuffle(self):
        self.deck = list(card_deck.keys()) * self.number_of_decks
        random.shuffle(self.deck)
        
    def deal(self, cards = 1):
        to_deal = self.deck[-cards:]
        self.deck = self.deck[:-cards]
        return to_deal
    
    
def new_game_clicked(b):
    clear_output(wait=True)
    display(HBox([number_of_players_input, players_ok_button]))
    
        
def players_ok_button_clicked(b):
    global number_of_players
    number_of_players = number_of_players_input.value
    new_game()
    
    
def new_game():
    global dealer
    global players
    dealer = Dealer()
    players = [Player(f"Player {player_number}") for player_number, player in enumerate(range(number_of_players), start=1)]
    betting()    
    

def betting():
    global players
    global bets_input
    clear_output(wait=True)
    if len(players) == 1:
        print("Place your bet:")
    else:
        print("Place your bets:")
    bets_input = []
    for player_number, player in enumerate(players, start=1):
        bets_input.append(
            BoundedIntText(
                value=10,
                min=1,
                max=player.money,
                step=1,
                disabled=False
            )
        )
        display(HBox([HTML(value=f"{player.name} ({player.money}$)"), bets_input[-1]]))
    display(betting_ok_button)
    
    
def betting_ok_button_clicked(b):
    global dealer
    global players
    global bets_input
    dealer.deck.shuffle()
    dealer.hand = Hand()
    dealer.hand.addToHand(dealer.deck.deal(2)) 
    if dealer.hand.value == 21:
         dealer.hand.blackjack = True
    for index, player in enumerate(players):
        player.hands = [Hand()]
        player.hands[0].bet = bets_input[index].value
        player.money -= player.hands[0].bet
        for i in range(2):
            player.hands[0].addToHand(dealer.deck.deal(1))
            clear_output(wait=True)
            print(f"{player.name}")
            display(HTML(player.hands[0].display()))
            time.sleep(short_pause)
        if player.hands[0].value == 21:
            player.hands[0].blackjack = True
            player.hands[0].finished_round = True
            print("Blackjack!!!")
            time.sleep(long_pause)
    game()
        
        
def game():
    global dealer
    global players
    global active_player
    global actice_hand
    
    for player_number, player in enumerate(players):
        for hand_number, hand in enumerate(player.hands):
            if hand.finished_round:
                continue
            else:
                active_player = player_number
                actice_hand = hand_number
                game_screen() 
                return None
    dealer_move()
        
    
def game_screen():
    global player_title_html
    player = players[active_player]
    hand = player.hands[actice_hand]
    clear_output(wait=True)
    buttons = []
    if hand.value == 21:
        buttons = [twenty_one_button]
    elif hand.value > 21:
        buttons = [bust_button]
    elif not hand.finished_round:
        buttons = [new_game_button, hit_button, stand_button]
        if len(hand.cards) == 2:
            if player.money >= hand.bet:
                buttons += [double_button]
                if card_deck[hand.cards[0]] == card_deck[hand.cards[1]]:
                    buttons += [split_button]
    
    player_title = f"{player.name} "
    if len(player.hands) > 1:
        player_title += f"Hand {actice_hand+1} "
    player_title_html = f"<p style='font-size: 16px; font-weight: bold; line-height: 16px;'>{player_title}</p>"     
    player_info_html = f"<p>Score({hand.value}) Money({player.money}$) Bet({hand.bet}$)</p>"
    player_html = f"<div>{player_title_html + player_info_html}</div>"
    
    display(
        GridBox(
            children=[
                HTML(value=dealer_title_html, layout=Layout(grid_area='dealer-title')),
                HTML(value=dealer.hand.display(1), layout=Layout(grid_area='dealer')),
                HTML(value=player_html, layout=Layout(grid_area='player-title')),
                HTML(value=players[active_player].hands[actice_hand].display(), layout=Layout(grid_area='player')),
                VBox(buttons, layout=Layout(grid_area='info'))
            ],
            layout=Layout(
                grid_template_columns='1fr 180px',
                grid_template_rows='repeat(4, auto)',
                grid_template_areas= '''
                    "dealer-title info"
                    "dealer info"
                    "player-title info"
                    "player info"
                    ''',
                grid_gap='8px')
            )
        )
        
    if len(buttons) < 2:
        time.sleep(long_pause)
        players[active_player].hands[actice_hand].finished_round = True
        game()

    
def hit_button_clicked(b):
    global dealer
    global players
    global active_player
    global actice_hand
    
    players[active_player].hands[actice_hand].addToHand(dealer.deck.deal(1))
    game()


def stand_button_clicked(b):
    global players
    global active_player
    global actice_hand
    
    players[active_player].hands[actice_hand].finished_round = True
    game()


def split_button_clicked(b):
    global dealer
    global players
    global active_player
    global actice_hand
    
    hand_to_split = players[active_player].hands.pop(actice_hand)
    bet = hand_to_split.bet
    players[active_player].money -= bet
    players[active_player].hands += [Hand(), Hand()]
    players[active_player].hands[-1].bet = bet
    players[active_player].hands[-2].bet = bet
    players[active_player].hands[-1].addToHand([hand_to_split.cards[0]] + dealer.deck.deal(1))
    players[active_player].hands[-2].addToHand([hand_to_split.cards[1]] + dealer.deck.deal(1))
    game()


def double_button_clicked(b):
    global dealer
    global players
    global active_player
    global actice_hand
    
    players[active_player].hands[actice_hand].finished_round = True
    players[active_player].money -= players[active_player].hands[actice_hand].bet
    players[active_player].hands[actice_hand].bet += players[active_player].hands[actice_hand].bet
    players[active_player].hands[actice_hand].addToHand(dealer.deck.deal(1))
    game_screen()
    

def new_round_button_clicked(b):
    betting()


def dealer_move():
    if dealer.hand.blackjack:
        dealer_move_screen(blackjack=True)
    else:
        dealer_move_screen()
        while dealer.hand.value < 17:
            time.sleep(short_pause)
            dealer.hand.addToHand(dealer.deck.deal(1))
            dealer_move_screen()
    time.sleep(long_pause)
    scoring()
            
    
def dealer_move_screen(blackjack = False):
    clear_output(wait=True)
    print(f"Dealer's move. Score ({dealer.hand.value})")
    display(HTML(value=dealer.hand.display()))
    if blackjack:
        print("Blackjack")
    elif dealer.hand.value > 21:
        print("BUST")
    
    
def scoring():
    global players
    players_are_bankrupt = True
    remaining_players = []
    for player_number, player in enumerate(players):
        for hand_number, hand in enumerate(player.hands):
            print(f"{player.name} ", end="")
            if len(player.hands) > 1:
                print(f"Hand {hand_number + 1} ", end="")
            bet = hand.bet
            if dealer.hand.blackjack: # dealer has blackjack
                if hand.blackjack: # dealer and player has blackjack
                    hand_push(player_number, bet)
                else: # dealer has blackjack but player doesn't
                    hand_minus_bet(player_number, bet)
            else: # dealer doesn't have blackjack
                if hand.blackjack: # player has blackjack
                    dealer.money -= round(1.5 * bet)
                    players[player_number].money += round(2.5 * bet)
                    print(f"+{round(1.5 * bet)}$")
                elif hand.value > 21: # player bust
                    hand_minus_bet(player_number, bet)
                elif dealer.hand.value > 21: # dealer bust
                    hand_plus_bet(player_number, bet)
                else:
                    if dealer.hand > hand:
                        player_minus_bet(player_number, bet)
                    elif dealer.hand < hand:
                        player_plus_bet(player_number, bet)
                    elif dealer.hand == hand:
                        player_push(player_number, bet)
        if player.money == 0:
            print(f"{player.name} is a bankrupt")
        else:
            players_are_bankrupt = False
            remaining_players.append(player)
    if players_are_bankrupt:
        print("Casinos always win, and this time is no different. Good luck next time")
        display(new_game_button)
    elif dealer.money <= 0:
        print("You beat the casino! You beat the game! Our congratulations!!")
        display(new_game_button)    
    else:
        print(f"Casino has {dealer.money}$")
        players = remaining_players
        display(new_round_button)
    
    
def player_plus_bet(player_number, bet):
    dealer.lose(bet)
    players[player_number].win(2 * bet)
    print(f"+{bet}$")
    
    
def player_minus_bet(player_number, bet):
    dealer.win(bet)
    print(f"-{bet}$")

    
def player_push(player_number, bet):
    players[player_number].money += bet
    print("Push")
    
    
new_game_button = Button(description="New game", layout=Layout(width='90%', max_width='250px', height='28px', margin='2px 2px 20px 2px'))
players_ok_button = Button(description="Ok", layout=Layout(width='80px', height='28px'))
betting_ok_button = Button(description="Ok", layout=Layout(width='80px', height='28px'))
number_of_players_input = BoundedIntText(value=1, min=1, max=7, step=1, description='Players:')
hit_button = Button(description="Hit", layout=Layout(width='90%', height='28px'))
stand_button = Button(description="Stand", layout=Layout(width='90%', height='28px'))
split_button = Button(description="Split", layout=Layout(width='90%', height='28px'))
double_button = Button(description="Double", layout=Layout(width='90%', height='28px'))
bust_button = Button(description="Bust", layout=Layout(width='90%', height='100%'))
bust_button.style.button_color = 'tomato'
twenty_one_button = Button(description="Twenty one", layout=Layout(width='90%', height='100%'))
twenty_one_button.style.button_color = 'mediumseagreen'
new_round_button = Button(description="New round", layout=Layout(width='100px', height='28px'))

new_game_button.on_click(new_game_clicked)
players_ok_button.on_click(players_ok_button_clicked)
betting_ok_button.on_click(betting_ok_button_clicked)
hit_button.on_click(hit_button_clicked)
stand_button.on_click(stand_button_clicked)
split_button.on_click(split_button_clicked)
double_button.on_click(double_button_clicked)
new_round_button.on_click(new_round_button_clicked)

display(new_game_button)

Button(description='New game', layout=Layout(height='28px', margin='2px 2px 20px 2px', max_width='250px', widt…

RULES:

Blackjack can play up to 7 players 

You need to place a bet before starting the game. Minimal bet is 1$

Win pays 1-to-1, blackjack pays 3-to-2

Dealer stand on soft 17

If your hand > 21, you bust. If two first cards on your hand = 21, you have the blackjack 

If you and the dealer have blackjack, you push

On your turn you can hit, stand, split or double. If you hit, you get a card. If you stand, the turn is passed further. You can split only if you have two cards with the same value. If you split, your hand is splitting in two hands and you continue playing them separately. If you double, you double your bet and get only one additional card.

Your goal is to win the casino by leaving them without money

If you have no money left, you lose